In [10]:
import pandas as pd

In [14]:
df = pd.read_json("test.jsonl", lines=True)

In [15]:
df["tps"] = (df["prompt_tokens"] + df["gen_tokens"]) / df["tps"]

In [16]:
df

,duration,prompt_tokens,gen_tokens,tps
0,4.680251,163,128,62.176151
1,5.031774,269,128,78.898610
2,5.525590,407,128,96.822242
3,6.491609,581,128,109.217914
4,6.672904,675,128,120.337410
5,7.213867,839,128,134.047375
6,8.586460,162,256,48.681297
7,8.807791,246,256,56.994993
8,9.328654,392,256,69.463400
9,10.079242,531,256,78.081270


In [17]:
overall_avg_duration = df['tps'].mean()
overall_percentile_99 = df['tps'].quantile(0.99)
overall_median = df['tps'].median()

print("Overall Average TPS:", overall_avg_duration)
print("99th Percentile TPS:", overall_percentile_99)
print("Median TPS:", overall_median)

Overall Average TPS: 68.16708048210872
99th Percentile TPS: 131.30538197545658
Median TPS: 60.35672438284156


In [20]:
# Calculate quartiles
gen_tokens_75th = df['gen_tokens'].quantile(0.75)
prompt_tokens_25th = df['prompt_tokens'].quantile(0.25)
prompt_tokens_75th = df['prompt_tokens'].quantile(0.75)
gen_tokens_25th = df['gen_tokens'].quantile(0.25)

# High gen_tokens with low prompt_tokens
high_gen_low_prompt = df[(df['gen_tokens'] >= gen_tokens_75th) & (df['prompt_tokens'] <= prompt_tokens_25th)]

# High prompt_tokens with low gen_tokens
high_prompt_low_gen = df[(df['prompt_tokens'] >= prompt_tokens_75th) & (df['gen_tokens'] <= gen_tokens_25th)]

# Define the interquartile range for both gen_tokens and prompt_tokens
gen_tokens_iqr = df['gen_tokens'].between(gen_tokens_25th, gen_tokens_75th)
prompt_tokens_iqr = df['prompt_tokens'].between(prompt_tokens_25th, prompt_tokens_75th)

# Balanced case: both gen_tokens and prompt_tokens fall within their respective IQRs
balanced_case = df[gen_tokens_iqr & prompt_tokens_iqr]


In [21]:
def calculate_metrics(dataframe):
    avg_duration = dataframe['tps'].mean()
    percentile_99 = dataframe['tps'].quantile(0.99)
    median = dataframe['tps'].median()
    return avg_duration, percentile_99, median

# Calculate metrics for each breakdown
high_gen_low_prompt_metrics = calculate_metrics(high_gen_low_prompt)
high_prompt_low_gen_metrics = calculate_metrics(high_prompt_low_gen)
balanced_case_metrics = calculate_metrics(balanced_case)

print("High Gen/Low Prompt Metrics:", high_gen_low_prompt_metrics)
print("High Prompt/Low Gen Metrics:", high_prompt_low_gen_metrics)
print("Balanced Case Metrics:", balanced_case_metrics)


High Gen/Low Prompt Metrics: (39.79981226123052, 40.68251010488449, 40.209861693430476)
High Prompt/Low Gen Metrics: (121.20089971050811, 133.77317572078013, 120.33740977401852)
Balanced Case Metrics: (66.19022578650589, 95.38835156683865, 60.35672438284156)
